In [2]:
from pycparser import c_parser, c_ast
import numpy as np
from sklearn.ensemble import RandomForestRegressor

class FeatureExtractor(c_ast.NodeVisitor):
    def __init__(self):
        self.n_loops = 0
        self.n_conditions = 0
        self.n_arithmetic_ops = 0
        # Add more features as needed

    def visit_For(self, node):
        self.n_loops += 1
        self.generic_visit(node)

    def visit_While(self, node):
        self.n_loops += 1
        self.generic_visit(node)

    def visit_If(self, node):
        self.n_conditions += 1
        self.generic_visit(node)

    def visit_BinaryOp(self, node):
        if node.op in ['+', '-', '*', '/']:
            self.n_arithmetic_ops += 1
        self.generic_visit(node)

def extract_features(code):
    # Wrap the code snippet in a function
    wrapped_code = f"""
    void func() {{
        {code}
    }}
    """
    parser = c_parser.CParser()
    try:
        ast = parser.parse(wrapped_code)
        extractor = FeatureExtractor()
        extractor.visit(ast)
        return [extractor.n_loops, extractor.n_conditions, extractor.n_arithmetic_ops]
    except:
        print(f"Failed to parse: {code}")
        return [0, 0, 0]  # Return default values if parsing fails

# Example code snippets and execution times
code_snippets = [
    "for(int i=0; i<100; i++) { a += i; }",
    "if(x > 0) { y = x * 2; }",
    "while(j < 50) { j++; }"
]
execution_times = [0.001, 0.0005, 0.0008]

# Extract features for all snippets
X = np.array([extract_features(code) for code in code_snippets])
y = np.array(execution_times)

# Train a Random Forest model
model = RandomForestRegressor()
model.fit(X, y)

# Predict execution time for a new snippet
new_code = "while(x < 100) { x *= 2; }"
new_features = extract_features(new_code)
predicted_time = model.predict([new_features])

print(f"Predicted execution time: {predicted_time[0]} seconds")

Predicted execution time: 0.000877333333333333 seconds


In [4]:
import clang.cindex

class FeatureExtractor:
    def __init__(self):
        self.features = {
            'if_statements': 0,
            'for_loops': 0,
            'while_loops': 0,
            'function_calls': 0,
            'variable_declarations': 0,
            'arithmetic_ops': 0,
            # Add more features here
        }

    def extract_features(self, node):
        if node.kind == clang.cindex.CursorKind.IF_STMT:
            self.features['if_statements'] += 1
        elif node.kind == clang.cindex.CursorKind.FOR_STMT:
            self.features['for_loops'] += 1
        elif node.kind == clang.cindex.CursorKind.WHILE_STMT:
            self.features['while_loops'] += 1
        elif node.kind == clang.cindex.CursorKind.CALL_EXPR:
            self.features['function_calls'] += 1
        elif node.kind == clang.cindex.CursorKind.VAR_DECL:
            self.features['variable_declarations'] += 1
        elif node.kind == clang.cindex.CursorKind.BINARY_OPERATOR:
            if node.binary_operator in [clang.cindex.BinaryOperator.Add, 
                                        clang.cindex.BinaryOperator.Subtract,
                                        clang.cindex.BinaryOperator.Multiply,
                                        clang.cindex.BinaryOperator.Divide]:
                self.features['arithmetic_ops'] += 1

        for child in node.get_children():
            self.extract_features(child)

def get_features(code):
    clang.cindex.Config.set_library_file(r'C:\Program Files\LLVM\bin\libclang.dll')
    index = clang.cindex.Index.create()
    tu = index.parse('temp.cpp', unsaved_files=[('temp.cpp', code)])
    extractor = FeatureExtractor()
    extractor.extract_features(tu.cursor)
    return extractor.features

# Example usage
code = """
int main() {
    int sum = 0;
    for (int i = 0; i < 100; i++) {
        if (i % 2 == 0) {
            sum += i;
        }
    }
    return sum;
}
"""

features = get_features(code)
print(features)

LibclangError: dlopen(C:\Program Files\LLVM\bin\libclang.dll, 0x0006): tried: 'C:\Program Files\LLVM\bin\libclang.dll' (no such file), '/System/Volumes/Preboot/Cryptexes/OSC:\Program Files\LLVM\bin\libclang.dll' (no such file), '/Users/vissan/anaconda3/envs/Cost/lib/python3.11/lib-dynload/../../C:\Program Files\LLVM\bin\libclang.dll' (no such file), '/Users/vissan/anaconda3/envs/Cost/bin/../lib/C:\Program Files\LLVM\bin\libclang.dll' (no such file), '/usr/lib/C:\Program Files\LLVM\bin\libclang.dll' (no such file, not in dyld cache), 'C:\Program Files\LLVM\bin\libclang.dll' (no such file), '/usr/local/lib/C:\Program Files\LLVM\bin\libclang.dll' (no such file), '/usr/lib/C:\Program Files\LLVM\bin\libclang.dll' (no such file, not in dyld cache). To provide a path to libclang use Config.set_library_path() or Config.set_library_file().